<center>
<img src="../../img/ods_stickers.jpg">
## Отворен курс по машинно обучение. Сесия №2
</center>
Автор на материала: Юрий Исаков и Юрий Кашницки. Материалът се разпространява съгласно условията на лиценза [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можете да го използвате за всякакви цели (редактиране, коригиране и използване като основа), освен търговски, но със задължителното споменаване на автора на материала.

# <center>Тема 4. Линейна класификация и регресионни модели
## <center> Практикувайте. Идентификация на потребителя чрез логистична регресия

Тук ще възпроизведем няколко бенчмарка от нашата конкуренция и ще се вдъхновим да победим третия бенчмарк, както и останалите участници. Тук няма да има уеб формуляр за изпращане на отговори, референтната точка е [табло за класиране](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session -tracking2/ класация) състезание.

In [4]:
import pickle

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm_notebook

%matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt

### 1. Зареждане и конвертиране на данни
Регистрирайте се в [Kaggle](www.kaggle.com), ако не сте го направили преди, отидете на [страница](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder - конкуренция за откриване-чрез-проследяване-на-сесия на уеб страница2) и изтегляне на данните. Първо, нека заредим примерите за обучение и тест и да разгледаме данните.

In [5]:
# тренировъчни и тестови проби за натоварване
train_df = pd.read_csv("../../data/train_sessions.csv", index_col="session_id")
test_df = pd.read_csv("../../data/test_sessions.csv", index_col="session_id")

# конвертиране на колони time1, ..., time10 във формат на време
times = ["time%s" % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# сортирайте данните по време
train_df = train_df.sort_values(by="time1")

# погледнете заглавката на примера за обучение
train_df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../../data/train_sessions.csv'

Учебният комплект съдържа следните характеристики:
    - site1 – индекс на първия посетен сайт в сесията
    - time1 – време на посещение на първия сайт в сесията
    - ...
    - site10 – индекс на 10-ия посетен сайт в сесията
    - time10 – време на посещение на 10-ия сайт в сесията
    - target – целева променлива, 1 за сесии на Alice, 0 за сесии на други потребители
    
Потребителските сесии се разпределят по такъв начин, че не могат да бъдат по-дълги от половин час или 10 сайта. Тоест, сесията се счита за приключила, когато потребителят е посетил 10 сайта подред или когато сесията е отнела повече от 30 минути.

В таблицата има липсващи стойности, което означава, че сесията се състои от по-малко от 10 сайта. Нека заменим липсващите стойности с нули и преобразуваме характеристиките в целочислен тип. Нека също да изтеглим речника на сайтовете и да видим как изглежда:

In [ ]:
# конвертирайте колоните site1, ..., site10 в целочислен формат и заменете пропуските с нули
sites = ["site%s" % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype("int")
test_df[sites] = test_df[sites].fillna(0).astype("int")

# заредете речника на сайтовете
with open(r"../../data/site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

# рамка с данни на речника на сайта
sites_dict_df = pd.DataFrame(
    list(site_dict.keys()), index=list(site_dict.values()), columns=["site"]
)
print(u"всего сайтов:", sites_dict_df.shape[0])
sites_dict_df.head()

Нека да изолираме целевата променлива и да комбинираме извадките, за да ги обединим в разреден формат.

In [ ]:
# нашата целева променлива
y_train = train_df["target"]

# таблица с обединени изходни данни
full_df = pd.concat([train_df.drop("target", axis=1), test_df])

# индекс, с който ще отделим обучителната от тестовата
idx_split = train_df.shape[0]

За първия модел ще използваме само посетени сайтове в сесията (но няма да обръщаме внимание на временни знаци). Идеята зад този избор на данни за модела е: *Алис има своите любими сайтове и колкото по-често виждате тези сайтове в сесия, толкова по-голяма е вероятността това да е сесията на Алис и обратното.*

Нека подготвим данните и от цялата таблица изберем само атрибутите `site1, site2, ..., site10`. Спомнете си, че липсващите стойности се заменят с нула. Ето как изглеждат първите редове на таблицата:

In [ ]:
# табела с индекси на посетените сайтове в сесията
full_sites = full_df[sites]
full_sites.head()

Сесиите са поредица от индекси на сайтове и данните в тази форма са неудобни за линейни методи. В съответствие с нашата хипотеза (Алиса има своите любими сайтове), трябва да трансформираме тази таблица по такъв начин, че всеки възможен сайт да има свой отделен атрибут (колона) и стойността му да е равна на броя посещения на този сайт в сесия. Това се прави на два реда:


In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
csr_matrix?

In [ ]:
# последователност с индекси
sites_flatten = full_sites.values.flatten()

# необходима матрица
full_sites_sparse = csr_matrix(
    (
        [1] * sites_flatten.shape[0],
        sites_flatten,
        range(0, sites_flatten.shape[0] + 10, 10),
    )
)[:, 1:]

Друго предимство на използването на разредени матрици е, че за тях има специални реализации както на матрични операции, така и на алгоритми за машинно обучение, което понякога позволява значително ускоряване на операциите поради особеностите на структурата на данните. Това важи и за логистичната регресия. Сега всички сме готови да изградим първия си модел.

### 2. Изграждане на първия модел

И така, имаме алгоритъм и данни за него, нека изградим нашия първи модел, използвайки версията [логистична регресия](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) от `package sklearn` с параметри по подразбиране. Ще използваме първите 90% от данните за обучение (наборът за обучение е сортиран по време), а останалите 10% за проверка на качеството (валидиране).

**Напишете проста функция, която ще върне качеството на модела на забавена извадка и обучете нашия първи класификатор**.

In [ ]:
def get_auc_lr_valid(X, y, C=1.0, ratio=0.9, seed=17):
"""
    X, y – образец
    съотношение – в какво съотношение да се раздели пробата
    C, seed – коефициент на регулация и random_state
              логистична регресия
    """

    # Вашият код е тук

**Вижте как се оказа ROC AUC на забавената проба.**

In [ ]:
# Вашият код е тук

Нека разгледаме този модел като нашата първа отправна точка (базова линия). За да се изгради модел за предсказване върху тестова извадка **необходимо е моделът да се обучи отново върху целия набор за обучение** (досега нашият модел беше обучен само върху част от данните), което ще увеличи способността му за обобщение:

In [ ]:
# функция для записи прогнозов в файл
def write_to_submission_file(
    predicted_labels, out_file, target="target", index_label="session_id"
):
    predicted_df = pd.DataFrame(
        predicted_labels,
        index=np.arange(1, predicted_labels.shape[0] + 1),
        columns=[target],
    )
    predicted_df.to_csv(out_file, index_label=index_label)

**Обучете модела върху цялата извадка, направете прогноза за тестовия набор и изпратете на състезанието**.

In [ ]:
# Вашият код е тук

Ако следвате тези стъпки и качите отговора на състезателната [страница](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) след това пуснете първия бенчмарк "Logit".

### 3. Подобряване на модела, изграждане на нови функции

Създайте знак, който ще бъде число от вида ГГГГММ от датата на провеждане на сесията, например 201407 - 2014 година и 7 месеца. По този начин ще вземем предвид месечната [линейна тенденция](http://people.duke.edu/~rnau/411trend.htm) за целия период на предоставените данни.

In [ ]:
# Вашият код е тук

Добавете нова функция, като преди това сте я мащабирали с помощта на `StandardScaler`, и отново изчислете ROC AUC върху забавената проба.

In [ ]:
# Вашият код е тук

**Добавете две нови функции: начален_час и сутрин.**

Атрибутът `start_hour` е часът, в който е започнала сесията (от 0 до 23), а двоичният атрибут `morning` е равен на 1, ако сесията е започнала сутринта и 0, ако сесията е започнала по-късно (ще приемем, че сутрин е, ако `начален_час е равен на` 11 или по-малко).

**Изчислете ROC AUC на забавена проба за проба с:**
- сайтове, `начален_месец` и `начален_час`
- сайтове, `начален_месец` и `сутрин`
- сайтове, `начален_месец`, `начален_час` и `сутрин`

In [ ]:
# Вашият код е тук

### 4. Избор на коефициент на регуляризация

И така, въведохме функции, които подобряват качеството на нашия модел в сравнение с първата базова линия. Можем ли да постигнем по-висока метрична стойност? След като сме формирали тренировъчната и тестовата извадка, почти винаги има смисъл да избираме оптимални хиперпараметри - характеристики на модела, които не се променят по време на обучението. Например, през седмица 3 сте преминали през решаващи дървета, дълбочината на дървото е хиперпараметър, но знакът, по който се извършва разклоняването, и неговата стойност не са. В логистичната регресия, която използваме, теглата на всяка характеристика се променят и техните оптимални стойности се намират по време на обучението, докато коефициентът на регулация остава постоянен. Това е хиперпараметърът, който сега ще оптимизираме.

Изчислете качеството на забавена проба с коефициент на регулиране, който по подразбиране е „C=1“:

In [ ]:
# Вашият код е тук

Ще се опитаме да победим този резултат чрез оптимизиране на коефициента на регулация. Нека вземем набор от възможни стойности на C и за всяка от тях изчислим метричната стойност на забавена проба.

Намерете „C“ от „np.logspace(-3, 1, 10)“, при което ROC AUC на забавената проба е максимизирана.

In [ ]:
# Вашият код е тук

Накрая, обучете модела с намерената оптимална стойност на коефициента на регулация и с конструираните характеристики `начален_час`, `начален_месец` и `сутрин`. Ако сте направили всичко правилно и сте изтеглили това решение, повторете втория бенчмарк от състезанието.

In [ ]:
# Вашият код е тук